### Expert Knowledge Worker

#### this is esseantially a question answering agent that is an expert knowledge worker

#### to be used by comployees of `InsureLLM`, an insurance Tech Startup.

#### the agent needs to be accurate and solution should be low cost.


I will be using RAG(Retrieval Augmented Generation) to ensure the question/answering assistant has high accuract

This is an implementation using vector database using langchain.

In [1]:
import os
from dotenv import load_dotenv
import glob
import gradio as gr

In [23]:
## langchain imports
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter # divide the text in smaller chunks
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_chroma import Chroma
from langchain.vectorstores import FAISS # Facebook AI Similarity Search
## in this project, I am using the CPU variant of FAISS. if need be, we can use the GPU variant as well.

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [4]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [5]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
## read the documents using LangChain's loader
## I will take everything from the sub-folder of the knowledge base


# get a list of folders in the knowledge-base folder
folders = glob.glob("knowledge-base/*")
print(folders)


# define the empty documents
documents = []

## iterate through each one of the folders
for folder in folders:
    ## get the document type, essentially the sub-folder name
    doc_type = os.path.basename(folder)
    print(doc_type)
    ## instantiate the directory loader
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader)
    ## load the data
    folder_docs = loader.load()

    ## setting the metadata for each document
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        ## extend the documents list
        documents.append(doc)


['knowledge-base/products', 'knowledge-base/contracts', 'knowledge-base/company', 'knowledge-base/employees']
products
contracts
company
employees


In [7]:
len(documents)

31

In [8]:
documents[0]

Document(metadata={'source': 'knowledge-base/products/Rellm.md', 'doc_type': 'products'}, page_content="# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility.\n\n## Features\n\n### AI-Driven Analytics\nRellm utilizes cutting-edge AI algorithms to provide predictive insights into risk exposures, enabling users to forecast trends and make informed decisions. Its real-time data analysis empowers reinsurance professionals with actionable intellige

In [9]:
## text splitter will split the documents into smaller chunks where each chunk will have 1000 characters with an overlap of 200 characters
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [10]:
len(chunks)

123

In [11]:
chunks[11]

Document(metadata={'source': 'knowledge-base/products/Homellm.md', 'doc_type': 'products'}, page_content="### 2. Dynamic Pricing Model\nWith Homellm's innovative dynamic pricing model, insurance providers can offer flexible premiums based on real-time risk evaluations and historical data. This adaptability ensures that customers pay a fair price that accurately reflects their unique risk profile.\n\n### 3. Instant Claim Processing\nThe AI-driven claims management system in Homellm automates the entire claims process, reducing processing time from weeks to hours. Insurers can resolve claims quickly and efficiently, leading to enhanced customer satisfaction.\n\n### 4. Predictive Maintenance Alerts\nHomellm incorporates predictive analytics to advise homeowners on potential risks and maintenance needs. By preventing issues before they arise, this feature helps customers minimize hazards, lowering the likelihood of claims.")

In [13]:
## cross checking if we have the right doc-types
doc_types = set([chunk.metadata['doc_type'] for chunk in chunks])
print(f"Document types: {doc_types}")
    

Document types: {'employees', 'contracts', 'products', 'company'}


In [14]:
print([chunk for chunk in chunks if "Lancaster" in chunk.page_content])
print("------")
print(len([chunk for chunk in chunks if "Lancaster" in chunk.page_content]))

[Document(metadata={'source': 'knowledge-base/company/about.md', 'doc_type': 'company'}, page_content="# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. It's first product was Markellm, the marketplace connecting consumers with insurance providers.\nIt rapidly expanded, adding new products and clients, reaching 200 emmployees by 2024 with 12 offices across the US."), Document(metadata={'source': 'knowledge-base/employees/Avery Lancaster.md', 'doc_type': 'employees'}, page_content='# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Av

### Auto-Encoding LLM

In [17]:
embeddings = OpenAIEmbeddings()

In [20]:
import shutil

## checking if the FAISS Datastore already exists. if yes, we need to delete that otherwise it will keep on adding the same data again and again
if os.path.exists(db_name):
    shutil.rmtree(db_name)

In [25]:
## create the vector datastore and populate

# vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
# print(f"Vectorstore created with {vectorstore._collection.count()} documents")

vectorstore = FAISS.from_documents(documents=chunks, embedding=embeddings)
print(f"Vectorstore created with {vectorstore.index.ntotal} documents")

Vectorstore created with 123 documents


In [ ]:
## get one vector and get the dimension of the vector
# collection = vectorstore._collection
# sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

dimensions = vectorstore.index.d
print(f"Embedding dimension: {dimensions}")

Embedding dimension: 1536


In [ ]:
## get a sample embedding from the FAISS vectorstore
sample_embedding = vectorstore.index.reconstruct(0)
print(sample_embedding)

[-0.01150598 -0.0144173  -0.00760565 ... -0.00124584 -0.02833313
 -0.0180669 ]


In [28]:
# ## visualize the Vector Store
# result = collection.get(include=["embeddings", "metadatas", "documents"])
# vectors = np.array(result["embeddings"])
# documents = result["documents"]
# doc_types = [metadata['doc_type'] for metadata in result["metadatas"]]
# colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

## pre-work for FAISS
vectors = []
documents = []
doc_types = []
colors = []
color_map = {'products': 'blue', 'employees': 'green', 'contracts': 'red', 'company': 'orange'}

for i in range(vectorstore.index.ntotal):
    vector = vectorstore.index.reconstruct(i)
    vectors.append(vector)
    
    doc_id = vectorstore.index_to_docstore_id[i]
    document = vectorstore.docstore.search(doc_id)
    documents.append(document.page_content)

    doc_type = document.metadata['doc_type']
    doc_types.append(doc_type)
    colors.append(color_map.get(doc_type, 'black'))  # Default to 'black'

vectors = np.array(vectors)

In [29]:
## turn the viz in 2D
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

## plotly viz
fig = go.Figure(data=[go.Scattergl(
    x=reduced_vectors[:,0],
    y=reduced_vectors[:,1],
    mode='markers',
    marker=dict(color=colors, size=5, opacity=0.8),
    text = [f"Type: {t}<br>Text: {doc[:100]}..." for t, doc in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title="2D Visualization of Document Embeddings", 
    xaxis_title="X axis", 
    yaxis_title="Y axis",
    width=800,
    height=600,
    margin=dict(l=10, r=20, b=10, t=40)
)

fig.show()

In [30]:
## 3d representation
tsne_3d = TSNE(n_components=3, random_state=42)
reduced_vectors_3d = tsne_3d.fit_transform(vectors)

fig_3d = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors_3d[:,0],
    y=reduced_vectors_3d[:,1],
    z=reduced_vectors_3d[:,2],
    mode='markers',
    marker=dict(color=colors, size=5, opacity=0.8),
    text = [f"Type: {t}<br>Text: {doc[:100]}..." for t, doc in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig_3d.update_layout(
    title="3D Visualization of Document Embeddings", 
    scene=dict(
        xaxis_title="X axis", 
        yaxis_title="Y axis",
        zaxis_title="Z axis"
    ),
    width=800,
    height=600,
    margin=dict(l=10, r=20, b=10, t=40)
)

fig_3d.show()

### Key Abstractions in LangChain
1. LLM: Abstraction of the model being used.
2. Retriever: Namely the database being used. In this case, I am using ChromaDB. This is essentially the vector data store.
3. Memory: Represent a form of history or discussion with the chatbot. In this case, we are using the traditional List of Dict. System, user, assistant, user, assistant....

### Steps involved to create a Conversation Chain with RAG and Memory
1. Create the LangChain LLM Object like ChatOpenAI
2. Create the Memory object like ConversationBufferMemory
3. Get the Retiever. We retrieve from the vectorstore being used.
4. Put these 3 abstraction all together and create the conversation chain.

In [31]:
## bringing everything together to create a chatbot with RAG and Memory
## abstraction 1: LLM
llm = ChatOpenAI(model=MODEL, temperature=0.7)

## Abstraction 2: Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

## Abstraction 3: Retriever
retriever = vectorstore.as_retriever()

## putting everything together to create the conversation chain
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/x7/xysp_t5n7pz6r9vs9x6sjsxm0000gn/T/ipykernel_5502/2670526494.py:6: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [32]:
## putting it to test
query = "can you describe InsureLLM in a few sentence?"
result = conversation_chain.invoke({"question": query})
print(result['answer'])

Insurellm is an innovative insurance tech startup founded by Avery Lancaster in 2015, focused on disrupting the insurance industry with cutting-edge products. With a workforce of 200 employees and 12 offices across the US, Insurellm offers four main software products: Carllm for auto insurance, Homellm for home insurance, Rellm for reinsurance, and Marketllm, a marketplace connecting consumers with insurance providers. The company serves over 300 clients worldwide, delivering advanced solutions to navigate the complexities of the insurance market.


In [33]:
## creating a chat interface using gradio


## creating the wrapper function for chat
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    answer = result['answer']
    return answer

In [34]:
view=gr.ChatInterface(fn=chat).launch(inbrowser=True)

/Users/protikmostafa/miniconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
